# Pinecone vector db

In [15]:
from langchain_openai import OpenAIEmbeddings
import os
from dotenv import load_dotenv

load_dotenv()

embeddings = OpenAIEmbeddings(
    model="text-embedding-3-small",
    dimensions=1024,
    api_key=os.environ["OPENAI_API_KEY"]
)

In [2]:
from pinecone import Pinecone
import os

pc = Pinecone(
    api_key=os.environ["PINECONE_API_KEY"]
)

In [ ]:
# from pinecone import ServerlessSpec

# # create index with model

# index_name = "rag-test"

# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
#         cloud="aws",
#         region="us-east-1",
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"}
#         }
#     )

In [ ]:
from pinecone import ServerlessSpec

# create index without model
index_name = "rag-test"

if not pc.has_index(index_name):
    pc.create_index(
        name=index_name,
        dimension=1024,
        metric="cosine",
        spec=ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
index = pc.Index(index_name)

In [6]:
index

In [18]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore(index=index, embedding=embeddings)

In [19]:
vector_store

In [20]:
from langchain_core.documents import Document

document_1 = Document(
    page_content="I had chocolate chip pancakes and scrambled eggs for breakfast this morning.",
    metadata={"source": "tweet"},
)

document_2 = Document(
    page_content="The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.",
    metadata={"source": "news"},
)

document_3 = Document(
    page_content="Building an exciting new project with LangChain - come check it out!",
    metadata={"source": "tweet"},
)

document_4 = Document(
    page_content="Robbers broke into the city bank and stole $1 million in cash.",
    metadata={"source": "news"},
)

document_5 = Document(
    page_content="Wow! That was an amazing movie. I can't wait to see it again.",
    metadata={"source": "tweet"},
)

document_6 = Document(
    page_content="Is the new iPhone worth the price? Read this review to find out.",
    metadata={"source": "website"},
)

document_7 = Document(
    page_content="The top 10 soccer players in the world right now.",
    metadata={"source": "website"},
)

document_8 = Document(
    page_content="LangGraph is the best framework for building stateful, agentic applications!",
    metadata={"source": "tweet"},
)

document_9 = Document(
    page_content="The stock market is down 500 points today due to fears of a recession.",
    metadata={"source": "news"},
)

document_10 = Document(
    page_content="I have a bad feeling I am going to get deleted :(",
    metadata={"source": "tweet"},
)

documents = [
    document_1,
    document_2,
    document_3,
    document_4,
    document_5,
    document_6,
    document_7,
    document_8,
    document_9,
    document_10,
]

documents

[Document(metadata={'source': 'tweet'}, page_content='I had chocolate chip pancakes and scrambled eggs for breakfast this morning.'),
 Document(metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(metadata={'source': 'tweet'}, page_content='Building an exciting new project with LangChain - come check it out!'),
 Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.'),
 Document(metadata={'source': 'tweet'}, page_content="Wow! That was an amazing movie. I can't wait to see it again."),
 Document(metadata={'source': 'website'}, page_content='Is the new iPhone worth the price? Read this review to find out.'),
 Document(metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.'),
 Document(metadata={'source': 'tweet'}, page_content='LangGraph is the best framework for building stateful, agentic application

In [21]:
vector_store.add_documents(documents)

['5af3a47c-1310-423b-ae6c-147145495747',
 '59cac637-8211-40dc-9e2f-afd1382abdcb',
 'b6ba9528-615d-4f7d-9e49-0d83659a80bb',
 '73b3eaf6-c1e2-4daf-b50c-537b9b34ac75',
 'a39f4587-f136-426b-8d5a-4fe28e20eeec',
 '2bf02ca7-c4c2-488b-9753-f09f4d9182e4',
 'e6a74635-31fe-4ba2-9172-67d82311141c',
 'cb448b56-6d11-4c6c-9ea5-27ee263208f4',
 'ffce934b-c113-4964-9652-7648cbb51f64',
 '551b2dd1-5fdb-4f6f-8612-a07d6685db2a']

In [22]:
# search from vector db

vector_store.similarity_search("what is the weather?")

[Document(id='59cac637-8211-40dc-9e2f-afd1382abdcb', metadata={'source': 'news'}, page_content='The weather forecast for tomorrow is cloudy and overcast, with a high of 62 degrees.'),
 Document(id='551b2dd1-5fdb-4f6f-8612-a07d6685db2a', metadata={'source': 'tweet'}, page_content='I have a bad feeling I am going to get deleted :('),
 Document(id='ffce934b-c113-4964-9652-7648cbb51f64', metadata={'source': 'news'}, page_content='The stock market is down 500 points today due to fears of a recession.'),
 Document(id='e6a74635-31fe-4ba2-9172-67d82311141c', metadata={'source': 'website'}, page_content='The top 10 soccer players in the world right now.')]

In [23]:
results = vector_store.similarity_search(
    "Langchain provides abstractions to make working with LLMs easy",
    k=3,
    filter={"source":"tweet"}
)

for res in results:
  print(f"* '{res.page_content}', metadata={res.metadata}")

* 'LangGraph is the best framework for building stateful, agentic applications!', metadata={'source': 'tweet'}
* 'Building an exciting new project with LangChain - come check it out!', metadata={'source': 'tweet'}
* 'Wow! That was an amazing movie. I can't wait to see it again.', metadata={'source': 'tweet'}


In [24]:
results = vector_store.similarity_search_with_score(
    "Langchain provides abstractions to make working with LLMs easy",
    k=3,
    filter={"source":"tweet"}
)

for res, score in results:
  print(f'* [SIM={score:.2f}] "{res.page_content}", metadata={res.metadata}')

* [SIM=0.43] "LangGraph is the best framework for building stateful, agentic applications!", metadata={'source': 'tweet'}
* [SIM=0.39] "Building an exciting new project with LangChain - come check it out!", metadata={'source': 'tweet'}
* [SIM=0.06] "Wow! That was an amazing movie. I can't wait to see it again.", metadata={'source': 'tweet'}


In [25]:
## Retriever

retriever = vector_store.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"k":1,
                   "score_threshold":0.5}
)

retriever.invoke("Stealing from bank is a crime",
                 filter={"source":"news"})

[Document(id='73b3eaf6-c1e2-4daf-b50c-537b9b34ac75', metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]

In [26]:
## Retriever

retriever = vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k":1}
)

retriever.invoke("Stealing from bank is a crime",
                 filter={"source":"news"})

[Document(metadata={'source': 'news'}, page_content='Robbers broke into the city bank and stole $1 million in cash.')]